In [ ]:
#install useful libraries
!pip install requests

In [ ]:
import requests

In [ ]:
# get url, the first part is all films without any filters
url="https://www.kinopoisk.ru/lists/movies/top250/?page=1"

r=requests.get(url)
r.text

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
# parse the data about each film using beautiful soup
soup=BeautifulSoup(r.text,'lxml')
print(soup)
print(type(soup))

In [ ]:
films=soup.findAll('div',class_='styles_root__ti07r')

data =[]
for film in films:
  name=film.find('a',class_='base-movie-main-info_link__YwtP1').find('span', class_='desktop-list-main-info_secondaryText__M_aus').text#.find('span', class_='desktop-list-main-info_secondaryText__M_aus')
  name_rus=film.find('div',class_='styles_content__nT2IG').find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text
  genre=film.find('div',class_='styles_content__nT2IG').findAll('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text
  rating_new=film.find('div',class_='styles_rating__ni2L0 styles_root___s7Tg styles_rootMd__ZvdRj styles_rootPositive__PIwO2').text
  rating_old=film.find('div',class_='styles_user__2wZvH').find('div',class_='styles_rating__LU3_x').find('span',class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg').text
  views=film.find('div',class_='styles_user__2wZvH').find('div',class_='styles_rating__LU3_x').find('span',class_='styles_kinopoiskCount__2_VPQ').text

  data.append([name,name_rus,genre,rating_new,rating_old,views])

In [ ]:
data

In [ ]:
import pandas as pd

In [ ]:
header =['name','name_rus','genre','rating_new','rating_old','views']

In [ ]:
df=pd.DataFrame(data,columns=header)
df.head()

In [ ]:
# convert pandas dataframe to a numpy array in order to perform calculations for Mann-Whitney test

x = df['rating_new']

In [ ]:
x = pd.to_numeric(x)

In [ ]:
y = df['rating_old']

In [ ]:
y = pd.to_numeric(x)

In [ ]:
import scipy.stats as stats

In [ ]:
rating_new = x.to_numpy()

In [ ]:
rating_old = y.to_numpy()

In [ ]:
rating_new

In [ ]:
rating_old

In [ ]:
import numpy as np

In [ ]:
rating_new = rating_new.astype(np.float)

In [ ]:
rating_new

In [ ]:
rating_old = rating_old.astype(np.float)

In [ ]:
rating_old

In [ ]:
#perform the Mann-Whitney U test
stats.mannwhitneyu(rating_new, rating_old, alternative='two-sided')

# here we het the test stat and the p-value. the hypothesis were the following 
#H0 - there is no statistical difference between rating_new and rating_old
#H1 - the difference between rating_new and rating_old exists

#we see the p-value equal to 1.0. 
#The higher the p-value, the more likely that we don’t reject the null hypothesis. 
#In this example, we got a 1.0, which means that we fail to reject H0 and, therefore, there is no statistical difference between ratings.

In [ ]:
#perform the prepartion for bootstrap statistical test
!pip install bstrap
from bstrap import bootstrap

means = []
new_means = []
old_means = []
for _ in range (10000):
    new_ratings = x.sample(50, replace=True).mean()
    old_ratings = y.sample(50, replace=True).mean()
    new_means.append(new_ratings)
    old_means.append(old_ratings)
    means.append(new_ratings - old_ratings)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#Lets plot this to compare the distributions
plt.hist(new_means, alpha=0.5);
plt.hist(old_means, alpha=0.5);

In [ ]:
#histogram of differences
plt.hist(means, alpha=0.5);

In [ ]:
# perform the boostrap statistical test
new_means, old_means, p_value = bootstrap(np.mean, x, y, nbr_runs=10000)
print(new_means)
print(old_means)
print(p_value)

#we see that p-value is vaery close to 1, so we cannot reject null hypothesis

In [ ]:
#perform all the same with the concrete country (USA)

url="https://www.kinopoisk.ru/lists/movies/top250/country--1/"

r=requests.get(url)
r.text

In [ ]:
soup=BeautifulSoup(r.text,'lxml')
print(soup)
print(type(soup))

In [ ]:
films=soup.findAll('div',class_='styles_root__ti07r')

data =[]
for film in films:
  name=film.find('a',class_='base-movie-main-info_link__YwtP1').find('span', class_='desktop-list-main-info_secondaryText__M_aus').text#.find('span', class_='desktop-list-main-info_secondaryText__M_aus')
  name_rus=film.find('div',class_='styles_content__nT2IG').find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text
  genre=film.find('div',class_='styles_content__nT2IG').findAll('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text
  rating_new=film.find('div',class_='styles_rating__ni2L0 styles_root___s7Tg styles_rootMd__ZvdRj styles_rootPositive__PIwO2').text
  rating_old=film.find('div',class_='styles_user__2wZvH').find('div',class_='styles_rating__LU3_x').find('span',class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg').text
  views=film.find('div',class_='styles_user__2wZvH').find('div',class_='styles_rating__LU3_x').find('span',class_='styles_kinopoiskCount__2_VPQ').text

  data.append([name,name_rus,genre,rating_new,rating_old,views])

In [ ]:
data

In [ ]:
df=pd.DataFrame(data,columns=header)
df.head()

In [ ]:
x = df['rating_new']
x = pd.to_numeric(x)
y = df['rating_old']
y = pd.to_numeric(x)
rating_new = x.to_numpy()
rating_old = y.to_numpy()

In [ ]:
stats.mannwhitneyu(rating_new, rating_old, alternative='two-sided')

In [ ]:
new_means, old_means, p_value = bootstrap(np.mean, x, y, nbr_runs=10000)
print(new_means)
print(old_means)
print(p_value)

In [ ]:
#the same with concrete genre (action movie)
url="https://www.kinopoisk.ru/lists/movies/top250/genre--action/"

r=requests.get(url)
r.text

In [ ]:
soup=BeautifulSoup(r.text,'lxml')
films=soup.findAll('div',class_='styles_root__ti07r')

data =[]
for film in films:
  name=film.find('a',class_='base-movie-main-info_link__YwtP1').find('span', class_='desktop-list-main-info_secondaryText__M_aus').text#.find('span', class_='desktop-list-main-info_secondaryText__M_aus')
  name_rus=film.find('div',class_='styles_content__nT2IG').find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text
  genre=film.find('div',class_='styles_content__nT2IG').findAll('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text
  rating_new=film.find('div',class_='styles_rating__ni2L0 styles_root___s7Tg styles_rootMd__ZvdRj styles_rootPositive__PIwO2').text
  rating_old=film.find('div',class_='styles_user__2wZvH').find('div',class_='styles_rating__LU3_x').find('span',class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg').text
  views=film.find('div',class_='styles_user__2wZvH').find('div',class_='styles_rating__LU3_x').find('span',class_='styles_kinopoiskCount__2_VPQ').text

  data.append([name,name_rus,genre,rating_new,rating_old,views])

In [ ]:
data

In [ ]:
df=pd.DataFrame(data,columns=header)
df.head()

In [ ]:
x = df['rating_new']
x = pd.to_numeric(x)
y = df['rating_old']
y = pd.to_numeric(x)
rating_new = x.to_numpy()
rating_old = y.to_numpy()

In [ ]:
stats.mannwhitneyu(rating_new, rating_old, alternative='two-sided')

In [ ]:
new_means, old_means, p_value = bootstrap(np.mean, x, y, nbr_runs=10000)
print(new_means)
print(old_means)
print(p_value)

In [ ]:
#the same with the concrete time period
url="https://www.kinopoisk.ru/lists/movies/top250/year--2000-2009/"

r=requests.get(url)
r.text

In [ ]:
soup=BeautifulSoup(r.text,'lxml')
films=soup.findAll('div',class_='styles_root__ti07r')

data =[]
for film in films:
  name=film.find('a',class_='base-movie-main-info_link__YwtP1').find('span', class_='desktop-list-main-info_secondaryText__M_aus').text#.find('span', class_='desktop-list-main-info_secondaryText__M_aus')
  name_rus=film.find('div',class_='styles_content__nT2IG').find('span', class_='styles_mainTitle__IFQyZ styles_activeMovieTittle__kJdJj').text
  genre=film.find('div',class_='styles_content__nT2IG').findAll('span', class_='desktop-list-main-info_truncatedText__IMQRP')[0].text
  rating_new=film.find('div',class_='styles_rating__ni2L0 styles_root___s7Tg styles_rootMd__ZvdRj styles_rootPositive__PIwO2').text
  rating_old=film.find('div',class_='styles_user__2wZvH').find('div',class_='styles_rating__LU3_x').find('span',class_='styles_kinopoiskValuePositive__vOb2E styles_kinopoiskValue__9qXjg').text
  views=film.find('div',class_='styles_user__2wZvH').find('div',class_='styles_rating__LU3_x').find('span',class_='styles_kinopoiskCount__2_VPQ').text

  data.append([name,name_rus,genre,rating_new,rating_old,views])

In [ ]:
df=pd.DataFrame(data,columns=header)
df.head()

In [ ]:
x = df['rating_new']
x = pd.to_numeric(x)
y = df['rating_old']
y = pd.to_numeric(x)
rating_new = x.to_numpy()
rating_old = y.to_numpy()

In [ ]:
stats.mannwhitneyu(rating_new, rating_old, alternative='two-sided')

In [ ]:
new_means, old_means, p_value = bootstrap(np.mean, x, y, nbr_runs=1000)
print(new_means)
print(old_means)
print(p_value)

In [ ]:
#we see that we cannot reject the null hypothesis in any selection (total, genre, country and time period)